In [34]:
# @title = "Error introduced by float32 and float16 from float64"

import numpy as np

number = np.random.rand(1)


print(number)
print(np.array(number, dtype=np.float32))
print(np.array(number, dtype=np.float16))


print(number-np.array(number, dtype=np.float32))
print(number-np.array(number, dtype=np.float16))

[0.47616784]
[0.47616783]
[0.476]
[1.36337572e-08]
[9.36227284e-05]


In [7]:
sorted(glob.glob('./emotion_recognition_data/*/eeg/sub-*_task-ImaginedEmotion_eeg.set'))[32:]


['./emotion_recognition_data/sub-34/eeg/sub-34_task-ImaginedEmotion_eeg.set',
 './emotion_recognition_data/sub-35/eeg/sub-35_task-ImaginedEmotion_eeg.set']

In [1]:
import glob
import mne
import numpy as np
import pandas as pd



# list files
files_name_eeglab = sorted(glob.glob('./emotion_recognition_data/*/eeg/sub-*_task-ImaginedEmotion_eeg.set'))[32:]
files_name_tsv = sorted(glob.glob('./emotion_recognition_data/*/eeg/sub-*_task-ImaginedEmotion_events.tsv'))[32:]

""" Experiment setup for the EEG emotion recognition study.

    source: https://openneuro.org/datasets/ds003004/versions/1.1.1

    PARADIGM: The study uses the method of guided imagery to induce resting, eyes-closed 
    participants using voice-guided imagination to enter distinct 15 emotion states during 
    acquisition of high-density EEG data.

    During the study, participants listen to 15 voice recordings that each suggest 
    imagining a scenario in which they have experienced -- or would experience the named 
    target emotion. Some target emotions have positive valence (e.g., joy, happiness), 
    others negative valence (e.g., sadness, anger). Before and between the 15 emotion 
    imagination periods, participants hear relaxation suggestions ('Now return to a neutral 
    state by ...').

    PROCEDURE: When the participant first begins to feel the target emotion, they are asked 
    to indicate this by pressing a handheld button. Participants are asked to continue 
    feeling the emotion as long as possible. To intensify and lengthen the periods of experienced 
    emotion, participants are asked to interoceptively perceive and attend relevant somatosensory 
    sensations. When the target feeling wanes (typically after 1 and 5 minutes), participants push 
    the button again to leave the emotion imagination period and cue the relaxation instructions.

"""


# pd.read_csv('./emotion_recognition_data/sub-01/eeg/sub-01_task-ImaginedEmotion_events.tsv', sep='\t')


#######################################################
######## LOAD THE ANNONTATIONS FROM THE TSV FILE ######
#######################################################

list_of_emotions = [ 
                    'awe', 
                    'frustration',
                    'joy',
                    'anger',
                    'happy',
                    'sad',
                    'love',
                    'fear',
                    'compassion',
                    'jealousy',
                    'content',
                    'grief',
                    'relief',
                    'excite',
                    'disgust'
                    ]

press_button = 'press1'


def get_emotion_onset(tsv_file, duration = 100.0):
    """ Get the onset of the emotion from the tsv file.

    Args:
        @tsv_file: the tsv file with the emotion data
        @duration: the duration of the emotion in seconds

    Returns:
        @df: a dataframe with the emotion, the onset and the duration
    
    """

    df = pd.read_csv(tsv_file, sep='\t')

    emotions = []
    onsents = []

    # we need to check if the next value is press1
    # when the person press1, means that he/she is 
    # feeling the emotion
    for idx, i in enumerate(zip(df['value'], df['onset'])):

        # check if the value is in the list of emotions
        if i[0] in list_of_emotions:  
            # check if the next value is press1
            if df['value'][idx+1] == press_button:

                # print(df['value'][idx], df['value'][idx+1], df['onset'][idx+1])

                # append the emotion
                emotions.append(df['value'][idx])

                # append the onset
                onsents.append(df['onset'][idx+1])

    # create a dataframe
    df = pd.DataFrame({'value': emotions, 'onset': onsents, 'duration': [duration]*len(emotions)})

    return df

#######################################################
######## LOAD EEG DATA FROM THE EEG FILE ##############
#######################################################

# load the data
def get_data_eeglab(files_name_eeglab, files_name_tsv, base_path):
    """ This function loads the data from the eeglab files and the tsv 
    files and saves them in a npz file.

    @param files_name_eeglab: list of strings
        list of the eeglab files
    @param files_name_tsv: list of strings
        list of the tsv files
    @param base_path: string
        path where to save the data

    @return: None
    """
    files_name = zip(files_name_eeglab, files_name_tsv)

    data = []
    for eeglab_file, tsv in files_name:

        #######################################################
        ######## LOAD EEG DATA FROM THE EEG FILE ##############
        #######################################################
        # load the data
        raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


        #######################################################
        ######## PERFORM ICA ##################################
        #######################################################

        # perform ica and plot
        # ica = mne.preprocessing.ICA(n_components=0.95, random_state=97, max_iter='auto')
        # ica.fit(raw)
        # ica_raw = ica.get_components()

        #######################################################
        ######## LOAD ANNOTATIONS FROM THE TSV FILE ###########
        #######################################################

        df = get_emotion_onset(tsv, duration = 100.0)

        # add the annotations to the raw data
        annot = mne.Annotations(
            onset=list(df['onset']),  # in seconds
            duration=list(np.array([60.0]*len(df['onset']))),  # in seconds, too
            description=list(df['value']),
        )

        # print the annotations
        # print(annot)

        raw2 = raw.copy().set_annotations(annot)

        # get events from the annotations
        events_from_annot, event_dict = mne.events_from_annotations(raw2)

        # epochs # 10 seconds before the onset and 90 seconds after the onset
        epochs = mne.Epochs(raw2, events_from_annot, event_id=event_dict, tmin=-10, tmax=90, preload=True)

        # apply baseline correction
        epochs.apply_baseline((None, 0))

        # filter the data
        epochs.filter(1, 40)

        # save the data
        print('SAVED: '+base_path + 'data_{}.npz'.format(eeglab_file.split('/')[2]))
        np.savez_compressed(base_path + 'data_{}.npz'.format(eeglab_file.split('/')[2]), data=np.array(epochs.get_data(),dtype='float32'))

        # save dataframe
        df.to_csv(base_path + 'data_{}.csv'.format(eeglab_file.split('/')[2]), index=False)


#######################################################


base_path = './data/emotion_recognition_preprocessed_data/'

get_data_eeglab(files_name_eeglab, files_name_tsv, base_path)

Reading /home/rickbook/document/neuro/project-abns/project-abns/experiments/emotion_recognition_data/sub-34/eeg/sub-34_task-ImaginedEmotion_eeg.fdt


/tmp/ipykernel_26638/104442116.py:132: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


Reading 0 ... 1108223  =      0.000 ...  4328.996 secs...
Used Annotations descriptions: ['anger', 'awe', 'compassion', 'content', 'disgust', 'excite', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'relief', 'sad']
Not setting metadata
15 matching events found
Setting baseline interval to [-10.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 15 events and 25601 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff fre

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 2700 out of 2700 | elapsed:    2.3s finished


SAVED: ./data/emotion_recognition_preprocessed_data/data_sub-34.npz
Reading /home/rickbook/document/neuro/project-abns/project-abns/experiments/emotion_recognition_data/sub-35/eeg/sub-35_task-ImaginedEmotion_eeg.fdt
Reading 0 ... 1511167  =      0.000 ...  5902.996 secs...


/tmp/ipykernel_26638/104442116.py:132: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(eeglab_file, preload=True)


Used Annotations descriptions: ['anger', 'awe', 'compassion', 'content', 'disgust', 'excite', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'relief', 'sad']
Not setting metadata
15 matching events found
Setting baseline interval to [-10.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 15 events and 25601 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 3120 out of 3120 | elapsed:    2.4s finished


SAVED: ./data/emotion_recognition_preprocessed_data/data_sub-35.npz
